In [1]:
from utils import *

from pathlib import Path
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

results_dir = Path("../results/split-freq-20k")
scalars = TBScalars(".cache/split_freq_20k")

In [3]:
dfs = []

freqs = [4, 2, 1]

fig = make_subplots(
    rows=len(freqs),
    row_titles=freqs,
    x_title="wm_freq",
    cols=len(freqs),
    column_titles=freqs,
    y_title="rl_freq",
    horizontal_spacing=0.1,
    vertical_spacing=0.05,
)

for row, wm_freq in enumerate(freqs):
    for col, rl_freq in enumerate(freqs):
        dfs = []
        for test_dir in results_dir.glob(f"*wm_freq={wm_freq}-rl_freq={rl_freq}*"):
            with open(test_dir / "config.yml", "r") as f:
                cfg = yaml.load(f)

            df = scalars.read(test_dir)
            if "tag" not in df.columns:
                continue
            df = df[df["tag"] == "val/mean_ep_ret"].copy()
            df["seed"] = cfg["repro"]["seed"]
            dfs.append(df)

        df = pd.concat(dfs, axis=0)

        xpr = px.line(df, x="step", y="value", color="seed")
        for trace in xpr.data:
            fig.add_trace(trace, row=row + 1, col=col + 1)

fig